In [2]:
import pandas as pd

In [3]:
column_types = {
    "Crash Date": str,
    "Crash Time": str,     
    "Borough": str,      
    "Zip Code": str,  
    "Latitude": float,
    "Longitude": float, 
    "Location": str,
    "On Street Name": str, 
    "Cross Street Name": str, 
    "Off Street Name": str,         
    "Numbers of Persons injured": int,      
    "Numbers of Persons killed": int,         
    "Number of Pedestrians injured": int,   
    "Number of Pedestrians killed": int,         
    "Number of cyclists injured": int,    
    "Number of cyclists killed": int,         
    "Number of motorists injured": int,   
    "Number of motorists killed": int,       
    "Contributing Factor Vehicle 1": str,       
    "Contributing Factor Vehicle 2": str,  
    "Contributing Factor Vehicle 3": str,      
    "Contributing Factor Vehicle 4": str,    
    "Contributing Factor Vehicle 5": str,
    "Collision ID": int,
    "Vehicle Type Code 1": str,
    "Vehicle Type Code 2": str,
    "Vehicle Type Code 3": str,
    "Vehicle Type Code 4": str,
    "Vehicle Type Code 5": str
}
crashes_dataset = pd.read_csv("collisions.csv", dtype=column_types, low_memory=False)

In [4]:
print(crashes_dataset)

# Als Ergebnis dann eine Datei mit bspw. Spalten für Lage, Straßenname und ein Ranking der Gefahr der Straße für Unfälle

         CRASH DATE CRASH TIME   BOROUGH ZIP CODE   LATITUDE  LONGITUDE  \
0        09/11/2021       2:39       NaN      NaN        NaN        NaN   
1        03/26/2022      11:45       NaN      NaN        NaN        NaN   
2        06/29/2022       6:55       NaN      NaN        NaN        NaN   
3        09/11/2021       9:35  BROOKLYN    11208  40.667202 -73.866500   
4        12/14/2021       8:13  BROOKLYN    11233  40.683304 -73.917274   
...             ...        ...       ...      ...        ...        ...   
1987316  04/25/2023      12:15       NaN      NaN  40.570835 -74.169830   
1987317  04/18/2023      21:16     BRONX    10475  40.868320 -73.833170   
1987318  04/19/2023       7:40       NaN      NaN  40.664303 -73.957300   
1987319  04/03/2023      15:50  BROOKLYN    11225  40.661568 -73.950730   
1987320  04/25/2023      19:05       NaN      NaN  40.748848 -73.872375   

                        LOCATION           ON STREET NAME CROSS STREET NAME  \
0                   